<a href="https://colab.research.google.com/github/AntiVini/QuotationUFRJ/blob/my-emailprovider/EmailProvider3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Challenge: Create a program that send emails to providers automatically.

Logical steps:


1.   Import all the worksheets of e-mails;
2.   Create a single worksheet that contains all the distinct e-mails;
3.   Create an e-mail to all my providers;
4.   Send e-mail.






Step 1 and 2: 

In [ ]:
import os.path
import pandas as pd


def email_distinct(path):
    if not os.path.isfile(r"/content/drive/MyDrive/Colab Notebooks/email_list.xlsx"):
        df = pd.DataFrame(columns=["e-mail"])
        df.to_excel(r"/content/drive/MyDrive/Colab Notebooks/email_list.xlsx")
    
    email_worksheet = pd.read_excel(path)  # Import database
    df1 = pd.read_excel(r"/content/drive/MyDrive/Colab Notebooks/email_list.xlsx")
    df1 = df1.drop(["Unnamed: 0"], axis=1)

    for email in email_worksheet["e-mail"]:  # Step 2: Create a worksheet with distinct e-mails.
        email = email.replace("['", "").replace("']", "")
        
        if email not in df1["e-mail"]:
            df1 = df1.append({'e-mail':email.replace("'", "")}, ignore_index=True)   
    
    df1.to_excel(r"/content/drive/MyDrive/Colab Notebooks/email_list.xlsx")

    pass


Step 3 and 4:

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib, ssl, mimetypes, email.mime.application, pandas as pd

def auto_send(process, path):
    msg = MIMEMultipart()
    port = 587
    smtp_server = "vmail.tic.ufrj.br"
    password = input("Type your password and press enter: ")
    me = "cotacao@pr6.ufrj.br"
    """to = exempleto@mail.com
    cc = exemplecopy@mail.com"""
    bcc = pd.read_excel(path)
    rcpt = list(bcc["e-mail"])
    msg['Subject'] = f"Solicitação de Proposta Comercial {process}"
    msg['From'] = me
    body = MIMEText(f"Prezados, \n\n    Venho por meio deste solicitar proposta comercial para prestação de serviços, conforme previsto no Termo de Referência em anexo.\n\n    As empresas interessadas deverão responder esse e-mail com a proposta comercial até o dia 30/03/2021.\n\n    Solicitamos que as propostas sejam enviadas nas seguintes especificações:\n\n    · A proposta deverá ser emitida em papel timbrado ou no modelo anexo;\n\n    · A proposta deverá ter informações do contato com telefone, endereço e correio eletrônico;\n\n    · A proposta deverá conter a data emissão;\n\n   · Validade da proposta: 60 dias;\n\n   · A proposta deverá ter o valor total do serviço já computando as despesas com os tributos previstos em lei;\n\n   · No valor de cada item deve estar contido todos os custos diretos e indiretos;\n\n   · A proposta deverá conter nº do CNPJ;\n\n   · A proposta deverá ter informação de pagamento contra empenho;\n\n   Obs.: A empresa vencedora deverá ter cadastro no SICAF ou ter as certidões negativa do INSS, FGTS, Receita Federal e Débitos Trabalhistas válidas.\n\n   Obs. 2: Para maiores informações entrar em contato através do correio eletrônico: cotacao@pr6.ufrj.br.\n\n   Caso sua empresa não tenha interesse em apresentar proposta para o objeto solicitado, peço a gentileza de enviar, via e-mail, as razões da recusa.\n\n   Atenciosamente,\n\n   Antonio Vinicius de Souza Alves\n\n   --\n\n   Universidade Federal do Rio de Janeiro\n   CNPJ 33.663.683/0001-16\n   Pró Reitoria de Gestão e Governança - PR-6\n   Rua Aloísio Teixeira, 278, Prédio 5, Parque Tecnológico - Cidade Universitária - RJ – CEP: 21941-850")
    msg.attach(body)

    # Attachment document
    file_path_tem = "/content/drive/MyDrive/Colab Notebooks/ProposalTemplate.docx"
    file_path_tr = "/content/drive/MyDrive/Colab Notebooks/termo de referencia.pdf"
    template = open(file_path_tem, 'rb')
    reference = open(file_path_tr, 'rb')
    attachment_tem = email.mime.application.MIMEApplication(template.read(), _subtype="docx")
    attachment_tr = email.mime.application.MIMEApplication(reference.read(), _subtype="pdf")
    template.close()
    reference.close()
    attachment_tem.add_header('Content-Disposition','attachment',filename="Modelo_de_Proposta.docx")
    attachment_tr.add_header('Content-Disposition','attachment',filename="Termo_de_Referência.pdf")
    msg.attach(attachment_tem)
    msg.attach(attachment_tr)

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.ehlo()  # Can be omitted
        server.starttls(context=context)
        server.ehlo()  # Can be omitted
        server.login(msg['From'], password)
        # rcpt = msg["Cc"].split(",") + msg["Bcc"].split(",") + [msg["To"]]
        server.sendmail(msg['From'], rcpt, msg.as_string())

    pass

# auto_send("23079.201396/2021-35", "/content/drive/MyDrive/Colab Notebooks/email_test.xlsx")